In [1]:
import ccxt
import pandas as pd
import datetime
import time

In [2]:
# Initialize the binance client
binance = ccxt.binance()

# Fetch all trading pairs
markets = binance.fetch_markets()

# Filter for USDT trading pairs
usdt_pairs = [market['symbol'] for market in markets if market['quote'] == 'USDT']

# Create an empty dataframe to stack the annual data
stack = pd.DataFrame()

# Get the current year
current_year = datetime.datetime.now().year

In [7]:
types = set(market['quote'] for market in markets)
print(types)

{'RUB', 'UST', 'BRL', 'GBP', 'VAI', 'PAX', 'USDP', 'BUSD', 'BKRW', 'ARS', 'XRP', 'AUD', 'UAH', 'ETH', 'IDRT', 'DAI', 'RON', 'BNB', 'DOGE', 'EUR', 'PLN', 'USDS', 'USDT', 'TRX', 'ZAR', 'TRY', 'BVND', 'FDUSD', 'TUSD', 'BIDR', 'USD', 'NGN', 'USDC', 'DOT', 'BTC'}


In [8]:
markets[0]

{'id': 'ETHBTC',
 'lowercaseId': 'ethbtc',
 'symbol': 'ETH/BTC',
 'base': 'ETH',
 'quote': 'BTC',
 'settle': None,
 'baseId': 'ETH',
 'quoteId': 'BTC',
 'settleId': None,
 'type': 'spot',
 'spot': True,
 'margin': True,
 'swap': False,
 'future': False,
 'option': False,
 'active': True,
 'contract': False,
 'linear': None,
 'inverse': None,
 'taker': 0.001,
 'maker': 0.001,
 'contractSize': None,
 'expiry': None,
 'expiryDatetime': None,
 'strike': None,
 'optionType': None,
 'precision': {'amount': 4, 'price': 5, 'base': 8, 'quote': 8},
 'limits': {'leverage': {'min': None, 'max': None},
  'amount': {'min': 0.0001, 'max': 100000.0},
  'price': {'min': 1e-05, 'max': 922327.0},
  'cost': {'min': 0.0001, 'max': 9000000.0},
  'market': {'min': 0.0, 'max': 3704.51422708}},
 'info': {'symbol': 'ETHBTC',
  'status': 'TRADING',
  'baseAsset': 'ETH',
  'baseAssetPrecision': '8',
  'quoteAsset': 'BTC',
  'quotePrecision': '8',
  'quoteAssetPrecision': '8',
  'baseCommissionPrecision': '8',
  '

In [5]:
len(usdt_pairs)

670

In [9]:
for pair in usdt_pairs:
    for year in range(2017, current_year+1):
        # Convert start and end dates to timestamps
        since = binance.parse8601(f'{year}-01-01T00:00:00Z')
        till = binance.parse8601(f'{year}-12-31T23:59:59Z')
        
        # Fetch the klines data
        klines = binance.fetch_ohlcv(pair, '1d', since, limit=(till-since)//(24*60*60*1000)+1)
        
        # Convert the klines data to a dataframe and append to stack
        headers = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
        df = pd.DataFrame(klines, columns=headers)
        
        # Convert the 'timestamp' column to a date format
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.rename(columns={'timestamp': 'date'}, inplace=True)
        
        # Add a column for the trading pair
        df['pair'] = pair
        
        stack = pd.concat([stack, df])
    
    # Introduce a delay to avoid hitting rate limits
    time.sleep(1) 

KeyboardInterrupt: 

In [12]:
stack

,date,open,high,low,close,volume,pair
0,2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,BTC/USDT
1,2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,BTC/USDT
2,2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,BTC/USDT
3,2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022,BTC/USDT
4,2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,BTC/USDT
...,...,...,...,...,...,...,...
3,2023-08-21,1705.94,1708.51,1687.14,1691.45,238.171000,ETH/USDT:USDT-231229
0,2023-08-18,1707.57,1720.86,1670.23,1688.18,351.950000,ETH/USDT:USDT-231229
1,2023-08-19,1688.02,1720.00,1677.51,1691.81,1275.527000,ETH/USDT:USDT-231229
2,2023-08-20,1691.89,1716.25,1684.12,1705.27,749.397000,ETH/USDT:USDT-231229


In [21]:
stack.to_csv("all_cryptos_usdt.csv", index=False)

In [3]:
data = pd.read_csv("all_cryptos_usdt.csv")

In [4]:
data.head()

,date,open,high,low,close,volume,pair
0,2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,BTC/USDT
1,2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,BTC/USDT
2,2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,BTC/USDT
3,2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022,BTC/USDT
4,2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,BTC/USDT
